# Create a tasking order via UP42 SDK

## Installation

[Install](https://sdk.up42.com/installation/) the SDK. The package requires Python ≥ 3.7.

In [1]:
!pip install up42-py
import up42

## Authentication

To [authenticate](https://sdk.up42.com/authentication), you need to know your project credentials.

In [2]:
up42.authenticate(
    project_id="your-project-ID",
    project_api_key="your-project-API-key"
)

# Alternatively, you can create a config.json file and use the following code for it:
# {
#     "project_id": your-project-ID",
#     "project_api_key": "your-project-API-key"
# }

# If you use a configuration file, uncomment this line:
# up42.authenticate(cfg_file="credentials.json")


## Step 1. Choose a tasking collection

Choose a tasking collection and get its `data_product_id` for ordering.

In [3]:
tasking = up42.initialize_tasking()
products = tasking.get_data_products(basic=True)
print(products)

Choose a data product and copy its ID.

In [4]:
# The following value is the product id for Pléiades Neo Tasking — Analytic (Mono):
data_product_id = "123eabab-0511-4f36-883a-80928716c3db"

## Step 2. Request access

If you want to order the chosen collection for the first time, you need to request access to it. For more information on access requests, see [Restrictions](https://docs.up42.com/getting-started/restrictions#tasking-collections).

An email from the Customer Success team usually takes up to 3 days. You can review your access request status on the [Access requests](https://console.up42.com/settings/access) page.

## Step 3. Accept a EULA

If you want to order the chosen collection for the first time, you need to accept its end-user license agreement (EULA). For more information on license agreements, see [EULAs](https://docs.up42.com/getting-started/account/eulas).

## Step 4. Get a JSON schema of an order form

Get detailed information about the parameters needed to create an order for the chosen data product.

In [5]:
tasking.get_data_product_schema(data_product_id)

## Step 5. Fill out an order form

Before placing an order, find out how order parameters affect the [timeframe of your tasking order](https://docs.up42.com/data/tasking/acquisition).

Specify geometry and use the required request body schema format for the chosen data product.

In [6]:
# geometry = up42.read_vector_file("data/aoi_washington.geojson")
# geometry = up42.get_example_aoi()

geometry = {
    "type": "Polygon",
    "coordinates": (
        (
            (13.375966, 52.515068),
            (13.375966, 52.516639),
            (13.378314, 52.516639),
            (13.378314, 52.515068),
            (13.375966, 52.515068),
        ),
    ),
}

order_parameters = tasking.construct_order_parameters(
    data_product_id=data_product_id,
    name="PNeo tasking order",
    acquisition_start="2023-11-01",
    acquisition_end="2023-12-20",
    geometry=geometry,
    tags=["project-7", "optical"]
)

The returned log will contain the required parameters you'll need to specify:
```
2023-04-27 16:54:24,397 - As `acquisitionMode` select one of ['mono']
2023-04-27 16:54:24,398 - As `cloudCoverage` select `{'description': 'Maximum allowed cloud coverage in percentage.', 'minimum': 5, 'maximum': 20, 'type': 'integer'}`
2023-04-27 16:54:24,399 - As `geometricProcessing` select one of ['ortho']
```

If needed, update the request by adding the requested parameters.

In [7]:
order_parameters["params"].update({
    "acquisitionMode": "mono",
    "cloudCoverage": 10,
    "incidenceAngle": 10,
    "geometricProcessing": "ortho",
    "spectralProcessing": "bundle",
    "pixelCoding": "12bits",
    "radiometricProcessing": "reflectance",
    "deliveredAs": "geotiff",
    "projection": "4326"
})

Check the parameters by printing them.

In [8]:
print(order_parameters)

## Step 6. Place an order

If multiple people need to be alerted about a future order, create a group email and [add it to the account](https://docs.up42.com/getting-started/account/management#change-your-email) you'll be placing an order with. Otherwise, only one person will receive notifications about it.

Place an order with the requested parameters.

In [9]:
order = tasking.place_order(order_parameters)
order

If you've defined an AOI with multiple geometries, each geometry will create a separate order. Order names will be suffixed with incrementing numbers: `Order 1`, `Order 2`, etc.

Once the order is placed, you will get a response like this:
```
2023-07-13 13:22:10,677 - Order is PLACED
2023-07-13 13:22:10,677 - Order your-order-id is now PLACED.
```

## Step 7. Review and accept feasibility

After an order is placed, the Operations team will conduct a [feasibility study](https://docs.up42.com/data/tasking/feasibility). They will evaluate the tasking parameters with the provider, and then will present the following assessments:

- The order is possible with the given parameters.
- The order requires modifications with suitable options proposed.

You will receive an email notifying you when your order feasibility has been assessed. Using your order ID, review the proposed feasibility options.

In [10]:
tasking.get_feasibility(order_id="68567134-27ad-7bd7-4b65-d61adb11fc78")

Accept one of the options.

In [11]:
tasking.choose_feasibility(
    feasibility_id="68567134-27ad-7bd7-4b65-d61adb11fc78",
    accepted_option_id="a0d443a2-41e8-4995-8b54-a5cc4c448227"
)

If none of the options are suitable, [contact](https://up42.com/company/contact-ordering) the Operations team.

You can't modify your order after accepting an option.

## Step 8. Activate an order

After selecting a feasibility option, you will receive a price quote for your order. Using your order ID, review the received quotation.

In [12]:
tasking.get_quotations(order_id="68567134-27ad-7bd7-4b65-d61adb11fc78")

To activate your order, accept the quotation.

In [13]:
tasking.decide_quotation(
    quotation_id="68567134-27ad-7bd7-4b65-d61adb11fc78",
    decision="ACCEPTED"
)

## Step 9. Monitor an order

Check the status of your order.

In [14]:
order.status

When the order is completed, [download its assets from storage](https://sdk.up42.com/storage/).